In [33]:
import numpy as np
import time
import aotools
import numpy as np
import scipy 
import astropy as ast
from itertools import islice
import psutil
import clr, os, winreg
import cv2
import random 


In [92]:

#np.random.shuffle(cn2)
N = 100
delta = 0.005
L0 = 10
l0 = 0.005
details=np.zeros((1,7))
details[0][0]=N #nx
details[0][1]=N #ny
details[0][2]=delta #dx
details[0][3]=delta #dy
details[0][4]=3 #unitflag
details[0][5]=0 #decenter_x
details[0][6]=0 #decenter_y
matrix = np.zeros((N*N, 4))



In [4]:
phi = np.zeros((100,100))
output = np.zeros((100,100))

In [35]:
class PythonStandaloneApplication(object):
    class LicenseException(Exception):
        pass
    class ConnectionException(Exception):
        pass
    class InitializationException(Exception):
        pass
    class SystemNotPresentException(Exception):
        pass

    def __init__(self, path=None):
        # determine location of ZOSAPI_NetHelper.dll & add as reference
        aKey = winreg.OpenKey(winreg.ConnectRegistry(None, winreg.HKEY_CURRENT_USER), r"Software\Zemax", 0, winreg.KEY_READ)
        zemaxData = winreg.QueryValueEx(aKey, 'ZemaxRoot')
        NetHelper = os.path.join(os.sep, zemaxData[0], r'ZOS-API\Libraries\ZOSAPI_NetHelper.dll')
        winreg.CloseKey(aKey)
        clr.AddReference(NetHelper)
        import ZOSAPI_NetHelper
        
        # Find the installed version of OpticStudio
        if path is None:
            isInitialized = ZOSAPI_NetHelper.ZOSAPI_Initializer.Initialize()
        else:
            # Note -- uncomment the following line to use a custom initialization path
            isInitialized = ZOSAPI_NetHelper.ZOSAPI_Initializer.Initialize(path)
        
        # determine the ZOS root directory
        if isInitialized:
            dir = ZOSAPI_NetHelper.ZOSAPI_Initializer.GetZemaxDirectory()
        else:
            raise PythonStandaloneApplication.InitializationException("Unable to locate Zemax OpticStudio.  Try using a hard-coded path.")

        # add ZOS-API referencecs
        clr.AddReference(os.path.join(os.sep, dir, "ZOSAPI.dll"))
        clr.AddReference(os.path.join(os.sep, dir, "ZOSAPI_Interfaces.dll"))
        import ZOSAPI

        # create a reference to the API namespace
        self.ZOSAPI = ZOSAPI

        # create a reference to the API namespace
        self.ZOSAPI = ZOSAPI

        # Create the initial connection class
        self.TheConnection = ZOSAPI.ZOSAPI_Connection()

        if self.TheConnection is None:
            raise PythonStandaloneApplication.ConnectionException("Unable to initialize .NET connection to ZOSAPI")

        self.TheApplication = self.TheConnection.CreateNewApplication()
        if self.TheApplication is None:
            raise PythonStandaloneApplication.InitializationException("Unable to acquire ZOSAPI application")

        if self.TheApplication.IsValidLicenseForAPI == False:
            raise PythonStandaloneApplication.LicenseException("License is not valid for ZOSAPI use")

        self.TheSystem = self.TheApplication.PrimarySystem
        if self.TheSystem is None:
            raise PythonStandaloneApplication.SystemNotPresentException("Unable to acquire Primary system")

    def __del__(self):
        if self.TheApplication is not None:
            self.TheApplication.CloseApplication()
            self.TheApplication = None
        
        self.TheConnection = None
    
    def OpenFile(self, filepath, saveIfNeeded):
        if self.TheSystem is None:
            raise PythonStandaloneApplication.SystemNotPresentException("Unable to acquire Primary system")
        self.TheSystem.LoadFile(filepath, saveIfNeeded)

    def CloseFile(self, save):
        if self.TheSystem is None:
            raise PythonStandaloneApplication.SystemNotPresentException("Unable to acquire Primary system")
        self.TheSystem.Close(save)

    def SamplesDir(self):
        if self.TheApplication is None:
            raise PythonStandaloneApplication.InitializationException("Unable to acquire ZOSAPI application")

        return self.TheApplication.SamplesDir

    def ExampleConstants(self):
        if self.TheApplication.LicenseStatus == self.ZOSAPI.LicenseStatusType.PremiumEdition:
            return "Premium"
        elif self.TheApplication.LicenseStatus == self.ZOSAPI.LicenseStatusTypeProfessionalEdition:
            return "Professional"
        elif self.TheApplication.LicenseStatus == self.ZOSAPI.LicenseStatusTypeStandardEdition:
            return "Standard"
        else:
            return "Invalid"
    
    def reshape(self, data, x, y, transpose = False):
        """Converts a System.Double[,] to a 2D list for plotting or post processing
        
        Parameters
        ----------
        data      : System.Double[,] data directly from ZOS-API 
        x         : x width of new 2D list [use var.GetLength(0) for dimension]
        y         : y width of new 2D list [use var.GetLength(1) for dimension]
        transpose : transposes data; needed for some multi-dimensional line series data
        
        Returns
        -------
        res       : 2D list; can be directly used with Matplotlib or converted to
                    a numpy array using numpy.asarray(res)
        """
        if type(data) is not list:
            data = list(data)
        var_lst = [y] * x;
        it = iter(data)
        res = [list(islice(it, i)) for i in var_lst]
        if transpose:
            return self.transpose(res);
        return res
    
    def transpose(self, data):
        """Transposes a 2D list (Python3.x or greater).  
        
        Useful for converting mutli-dimensional line series (i.e. FFT PSF)
        
        Parameters
        ----------
        data      : Python native list (if using System.Data[,] object reshape first)    
        
        Returns
        -------
        res       : transposed 2D list
        """
        if type(data) is not list:
            data = list(data)
        return list(map(list, zip(*data)))

if __name__ == '__main__':
    zos = PythonStandaloneApplication()
    
    # load local variables
    ZOSAPI = zos.ZOSAPI
    TheApplication = zos.TheApplication
    TheSystem = zos.TheSystem
    sampleDir = TheApplication.SamplesDir
    
    zmxfile = 'DataCollector.zmx'
    lensPath = os.path.join(os.getcwd())
    testFile = os.path.join(lensPath, zmxfile)      
    
    TheSystem.LoadFile(testFile, False)
    #TheSystem.SaveAs(testFile)    

Exception ignored in: <function PythonStandaloneApplication.__del__ at 0x000002DAEB45B678>
Traceback (most recent call last):
  File "<ipython-input-32-74e99755eebf>", line 63, in __del__
System.Runtime.Remoting.RemotingException: Failed to connect to an IPC Port: The system cannot find the file specified.
 

Server stack trace: 
   at System.Runtime.Remoting.Channels.Ipc.IpcPort.Connect(String portName, Boolean secure, TokenImpersonationLevel impersonationLevel, Int32 timeout)
   at System.Runtime.Remoting.Channels.Ipc.ConnectionCache.GetConnection(String portName, Boolean secure, TokenImpersonationLevel level, Int32 timeout)
   at System.Runtime.Remoting.Channels.Ipc.IpcClientTransportSink.ProcessMessage(IMessage msg, ITransportHeaders requestHeaders, Stream requestStream, ITransportHeaders& responseHeaders, Stream& responseStream)
   at System.Runtime.Remoting.Channels.BinaryClientFormatterSink.SyncProcessMessage(IMessage msg)

Exception rethrown at [0]: 
   at System.Runtime.Remoti

In [93]:
    
    start_time = time.time()
    
    #########################################   Setting up Zemax   ################################################
    
    TheLDE = TheSystem.LDE
    
    # Making surface 2 as Stop
    TheLDE.InsertNewSurfaceAt(2)
    TheSystem.LDE.GetSurfaceAt(2).Thickness = 5000000
    TheSystem.LDE.GetSurfaceAt(2).IsStop = True
    analysis = TheSystem.Analyses.New_Analysis(ZOSAPI.Analysis.AnalysisIDM.PhysicalOpticsPropagation)
    analysisSettings = analysis.GetSettings()    
    
    # Temp file
    cfgFile = os.path.join(lensPath, 'sha.CFG')

    # Save the current settings to the temp file
    analysisSettings.SaveTo(cfgFile)

    # modify parameters, Caution!!! Change with care
    analysisSettings.ModifySettings(cfgFile, 'POP_END', '3')# Surface 3
    analysisSettings.ModifySettings(cfgFile, 'POP_BEAMTYPE', '0')# TopHat = 3, Gaussian = 0
    analysisSettings.ModifySettings(cfgFile, 'POP_PARAM1', '100')# X Waist
    analysisSettings.ModifySettings(cfgFile, 'POP_PARAM2', '100')# Y Waist
    analysisSettings.ModifySettings(cfgFile, 'POP_SAMPX','6')
    analysisSettings.ModifySettings(cfgFile, 'POP_SAMPX','6')
    analysisSettings.ModifySettings(cfgFile, 'POP_WIDEX','4000')# X-Width
    analysisSettings.ModifySettings(cfgFile, 'POP_WIDEY','4000')# Y-Width
    
    # load the temp file
    analysisSettings.LoadFrom(cfgFile) 
        


    
    
     ########################################  Different Phasescreens   ##########################################
    for p in range(1):
        logcn2 = np.zeros(0)
        
        for i in range(5):
            rand = int.from_bytes(os.urandom(8), byteorder="big") / ((1 << 64) - 1)
            logcn2 = np.append(logcn2,-13*(1-rand)-16*rand) 
        cn2 = 10**(logcn2)
        
        f=open('D:\\Nivedita\\Data\\Recheck\\cn2_'+str(p+8)+'.DAT','w')
        np.savetxt(f,cn2)
        f.close()
        
        # Phasescreen data stored in f1 #
        f1 = open('D:\\Nivedita\\Data\\Recheck\\phasescrn_'+ str(p+8)+'.DAT','w')
    
        # Image data stored in f2 #
        f2 = open('D:\\Nivedita\\Data\\Recheck\\image_'+ str(p+8)+'.DAT','w')
        for i in range(5):
            
            # Find fried parameter #
            r0 = aotools.turbulence.atmos_conversions.cn2_to_r0(cn2[i], lamda=5.5e-07)
            #print(r0)
            # j = number of phasescreens #
            for j in range(1):

                TheLDE.InsertNewSurfaceAt(j+1)
                

                # Phase screen generation 
                phi= aotools.turbulence.phasescreen.ft_phase_screen(r0, N, delta, L0, l0, FFT = None, seed = random.randint(0,5000))
                
                file = open('phscrn'+str(j)+'.DAT', 'ab')
                
                #  File used in zemax has first row with details and then 4 cols with 1st col having (N*N) 
                #  size with phi as values.
                #  The other three columns (3 double derivatives wrt x and y) being zero 

                file.truncate(0)
                np.savetxt(file,details)# firstrow
                k = 0
                for m in range(N):# firstcolumn
                    for n in range(N):
                        matrix[k][0] = phi[m][n]
                        k = k+1
                np.savetxt(file,matrix)
                file.close()

                # Zemax getting surface and Importing Gridphase
                Surface = TheLDE.GetSurfaceAt(j+1)
                SurfaceType_Gridphase = TheLDE.GetSurfaceAt(1+j).GetSurfaceTypeSettings(ZOSAPI.Editors.LDE.SurfaceType.GridPhase)
                Surface.ChangeType(SurfaceType_Gridphase)
                Surface.Thickness = 5000000.0
                
                
                phasefile = 'phscrn'+str(j)+'.DAT'
                phasescreen = os.path.join(lensPath, phasefile)
                Surface.ImportData.ImportDataFile(phasescreen)
                
                
            # We are using same analysis settings for all iterations, so we just apply and wait 
            analysis.ApplyAndWaitForCompletion()
            
            # Extracting Result 
            res=analysis.GetResults() 
            imgarr=res.GetDataGrid(0).Values
            img = zos.reshape(imgarr, imgarr.GetLength(0), imgarr.GetLength(1))
            img = np.flipud(img)
            
            # Normalizing to get same image size 
            if(50*p+i==0):
                dx0=res.GetDataGrid(0).Dx
                output = img[512-50:512+50,512-50:512+50]
                
            Nx = 100*dx0/res.GetDataGrid(0).Dx
            output = img[512-int(Nx):512+int(Nx),512-int(Nx):512+int(Nx)]
            output = cv2.resize(output, (100,100))
            
            # Saving text p.s. No delimiters between two phasescreens 
            np.savetxt(f1,phi)
            np.savetxt(f2,output)
            for j in range(1): # j is the number of phasescreens 
                TheLDE.RemoveSurfaceAt(j+1)
        # Closing files
        f1.close()
        f2.close()
    TheLDE.RemoveSurfaceAt(2)    
    analysis.Close()
    TheSystem.Save()
    
    print("--- %s seconds ---" % (time.time() - start_time))

--- 10.176784753799438 seconds ---


In [152]:
    ###Close Zemax#####
    del zos
    zos = None  